<a href="https://colab.research.google.com/github/rigonzal/ML-2019-1/blob/master/Tarea3/Tarea3_Pregunta1_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INF-393: Máquinas de Aprendizaje

## Tarea 3 - Pregunta 1

13-09-2019



*   Rodrigo González Smith           -- 201303026-2
*   Ignacio Valenzuela Albornoz   -- 201473055-1


## Regresión para ubicación espacial

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings

from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

drive.mount('/content/drive')

%matplotlib inline

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## a) Carga de datos

En esta ocasión, realizaremos un análisis en un dataset bien particular. El problema en cuestión tiene relación con la ubicación espacial de ciertos dispositivos en lugares cerrados, principalmente en edificaciones. En espacios exteriores tenemos la señal GPS con la que nuestros celulares y otros dispositivos detectan nuestra ubicación, pero en interiores el material de las estructuras reducen o incluso reflectan estas señales, por lo que se han buscado diversos métodos o técnicas para poder modelar la ubicación espacial sin la ayuda del GPS.

El dataset es de la Universidad de Jaume, el cual se hicieron 20000 mediciones en 3 estructuras de esta universidad. Primero que todo, cargaremos los datos desde la nube en google drive e imprimimos las primeras 5 filas para identificar la composición del dataset.

In [0]:
training = pd.read_csv("/content/drive/My Drive/trainingData.csv")
validation = pd.read_csv("/content/drive/My Drive/validationData.csv")

In [0]:
training[:5]

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,WAP013,WAP014,WAP015,WAP016,WAP017,WAP018,WAP019,WAP020,WAP021,WAP022,WAP023,WAP024,WAP025,WAP026,WAP027,WAP028,WAP029,WAP030,WAP031,WAP032,WAP033,WAP034,WAP035,WAP036,WAP037,WAP038,WAP039,WAP040,...,WAP490,WAP491,WAP492,WAP493,WAP494,WAP495,WAP496,WAP497,WAP498,WAP499,WAP500,WAP501,WAP502,WAP503,WAP504,WAP505,WAP506,WAP507,WAP508,WAP509,WAP510,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-92,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710


Podemos apreciar que posee una cantidad de atributos inmensa donde la mayoría no parece darnos información muy explícita, excepto las últimas columnas que tienen nombres más intuitivos. Aún así, revisaremos la documentación del dataset para conocer mejor qué significa cada atributo en cuestión.



*   WAP: Este atributo se repite unas 520 veces. WAP significa "Wi-Fi Access Point", que representa los puntos de acceso a WiFi que posee la Universidad. El valor de cada WAP representa la intensidad que esta posee, con valores enteros en un rango de -104 a 0. También puede tomar el valor de 100 que indica que no se detectó, en esa medición, el punto de acceso WAPxxx.
*   Longitude: Es la longitud de donde se realizó la medición. Toma valores negativos entre -7695.9387549299299000 to -7299.786516730871000.
*   Latitude: La latitud de donde se realizó la medición. Toma valores positivos entre 4864745.7450159714 to 4865017.3646842018
*   Floor: Un valor entero entre 0 y 4, que indica el piso donde se hizo la medición. También se puede considerar como la altitud de la medición
*   BuildingID: Las mediciones se realizaron en 3 edificios diferentes de la universidad, por lo tanto se representa en la medición en valores enteros de 0, 1 y 2.
*   SpaceID: Un identificador es específica en que tipo de espacio se realizó la medición. 0 para una oficina, 1 para un pasillo y 2 para una sala de clases.
*   RelativePosition: Refleja la posición relativa de la posición. Posee un valor booleano, donde 0 implica que se está dentro (de la oficina o sala de clases por ejemplo) y 1 que se encuentra fuera de esta pero en frente de la puerta de entrada.
*   UserID: Es el identificador del usuario que realizó la medición. Es un valor categórico, por lo que su representación se puede ver a continuación:
 

      0 USER0000 (Validation User) N/A
      1 USER0001 170
      2 USER0002 176
      3 USER0003 172
      4 USER0004 174
      5 USER0005 184
      6 USER0006 180
      7 USER0007 160
      8 USER0008 176
      9 USER0009 177
      10 USER0010 186
      11 USER0011 176
      12 USER0012 158
      13 USER0013 174
      14 USER0014 173
      15 USER0015 174
      16 USER0016 171
      17 USER0017 166
      18 USER0018 162 

*   PhoneID: Es el identificador del celular que se utilizó para la medición. También es categórico y representa principalmente el modelo de celular y su versión de android. Las representaciones son como siguen:

      0 Celkon A27 4.0.4(6577) 0
      
      1 GT-I8160 2.3.6 8
      
      2 GT-I8160 4.1.2 0
      
      3 GT-I9100 4.0.4 5
      
      4 GT-I9300 4.1.2 0
      
      5 GT-I9505 4.2.2 0
      
      6 GT-S5360 2.3.6 7
      
      7 GT-S6500 2.3.6 14
      
      8 Galaxy Nexus 4.2.2 10
      
      9 Galaxy Nexus 4.3 0
      
      10 HTC Desire HD 2.3.5 18
      
      11 HTC One 4.1.2 15
      
      12 HTC One 4.2.2 0
      
      13 HTC Wildfire S 2.3.5 0,11
      
      14 LT22i 4.0.4 0,1,9,16
      
      15 LT22i 4.1.2 0
      
      16 LT26i 4.0.4 3
      
      17 M1005D 4.0.4 13
      
      18 MT11i 2.3.4 4
      
      19 Nexus 4 4.2.2 6
      
      20 Nexus 4 4.3 0
      
      21 Nexus S 4.1.2 0
      
      22 Orange Monte Carlo 2.3.5 17
      
      23 Transformer TF101 4.0.3 2
      
      24 bq Curie 4.1.1 12 


*   TimeStamp: Representa la hora, en formato timestamp, en la cual fue realizada la medición.


Por el momento, combinaremos ambos archivos en un solo dataframe para poder manejar más a nuestro antojo la división entre conjuntos de entrenamiento y validación.

In [0]:
df = training.append(validation, ignore_index = True).sample(frac = 1).reset_index().drop("index", axis = 1)

In [0]:
targets = np.array([520,521,522])
attributes = np.arange(0, len(df.columns))
target_col =  df.columns[targets]  # Obtenemos las columnas que representarán el target
attribute_col = df.columns[np.setdiff1d(attributes, targets)] # Rescatamos las columnas que no son target
x_un = pd.get_dummies(df.PHONEID) # Categorizamos el atributo phoneid
categorical_columns = x_un.columns # Almacenamos el nombre de los nuevos atributos dummy
x_un[attribute_col] = df[attribute_col] # Finalmente agregamos las demás columnas que nos ayudan en el target
y_u = df[target_col] #Finalmente tenemos otro dataframe con los targets

Ya tenemos el dataset preparado, y ahora debemos volver a dividirlo en training y validation. Debido a que el dataset ya fue aleatoriamente ordenado, podemos dividirlo respecto a sus índices solamente (en vez de usar train_test_split, por ejemplo). Una de las incidencias que podría ocurrir al no hacer shuffle es que las mediciones se hayan hecho y almacenado de forma consecutiva, por lo que habría un factor de tiempo involucrado, por lo que muchas filas juntas pueden tener valores de latitud y longitud más cercanas, en cambio si realizamos un shuffle rompemos esa relación, teniendo así solos los atributos como fuentes de información para definir la latitud o longitud en el cual ze encontraba el dispositivo en el momento de la medición.


In [0]:
x_tr_un = x_un[0:16838]
y_tr_un = y_u[0:16838]

x_val_un = x_un[16838:]
y_val_un = y_u[16838:]

## b) Estandarizar los datos

Ahora se estandarizarán los datos. Debido a que poseemos datos categóricos y no categóricos, se debe aplicar una estandarización distinta. Respecto a los atributos no categóricos, aplicaremos un StandardScaler que estandarizará respecto a la media y desviación estándar de cada atributo. Por otra parte, los atributos categóricos (que son 0 o 1) se les restará 0.5, para que así recorran tanto el espacio de los negativos y los positivos y estén centrados en 0.

También debemos estandarizar los targets, que son la latitud, longitud y los pisos. El atributo floor es categórico, pero aún así se considerará como continuo (se le aplicará StandardScaler) para así ver como nuestros modelos pueden aproximar la altitud de las mediciones.

In [0]:
# Datos de entrenamiento

scaler_x = StandardScaler()
scaler_x.fit(x_tr_un[attribute_col])
x_tr = pd.np.concatenate((scaler_x.transform(x_tr_un[attribute_col]), x_tr_un[categorical_columns].values-0.5), axis=1)

scaler_y = StandardScaler()
scaler_y.fit(y_tr_un)
y_tr = pd.DataFrame(scaler_y.transform(y_tr_un))

# Datos de validacion

scaler_x = StandardScaler()
scaler_x.fit(x_val_un[attribute_col])
x_val = pd.np.concatenate((scaler_x.transform(x_val_un[attribute_col]), x_val_un[categorical_columns].values-0.5), axis=1)

scaler_y = StandardScaler()
scaler_y.fit(y_val_un)
y_val = pd.DataFrame(scaler_y.transform(y_val_un))

In [0]:
y_tr_o = []
y_val_o = []
for index, row in y_tr.iterrows():
  y_tr_o.append(np.array(row))
for index, row in y_val.iterrows():
  y_val_o.append(np.array(row))
  
y_tr = np.array(y_tr_o)
y_val = np.array(y_val_o)

## c) Primer árbol

Comenzaremos con la construcción de un árbol de profundidad 3 y con una función de error MAE (Maximum Absolute Error). Mediremos su desempeño, tiempo de ejecución y comparativas con un modelo de regresión lineal común.

In [0]:
tree = DecisionTreeRegressor(criterion='mae', max_depth=3)
ini = time.time()
tree.fit(x_tr, y_tr)
fin = time.time()
print(fin - ini, "[s]")

79.79273533821106 [s]


Vemos que el entrenamiento del árbol de decisión se demoró aproximadamente minuto y medio. Calculemos el score que nos ofrece el árbol en ambos conjuntos de datos.

In [0]:
score_tr = tree.score(x_tr, y_tr)
score_val = tree.score(x_val, y_val)

print("Training set score: ", score_tr)
print("Validation set score: ", score_val)

predicted_tr = tree.predict(x_tr)
predicted_val = tree.predict(x_val)
MAE_tr = mean_absolute_error(y_tr, predicted_tr)
MAE_val = mean_absolute_error(y_val, predicted_val)

print("Training set MAE: ", MAE_tr)
print("validation set MAE: ", MAE_val)

Training set score:  0.671262329073306
Validation set score:  0.6680176057709891
Training set MAE:  0.3731321734914379
validation set MAE:  0.38143475694074036


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

Podemos apreciar que el score en ambos conjuntos es bastante parecido, un 67% aproximadamente. Vimos que el tiempo de entrenamiento fue mas de lo que acostumbramos en el curso, por lo que comprobemos con una regresión lineal común si podemos obtener un desempeño parecido o mejor con menos tiempo.

In [0]:
ini = time.time()
lin_reg = LinearRegression()
lin_reg.fit(x_tr, y_tr)
fin = time.time()
print(fin - ini, "[s]")

1.09576416015625 [s]


In [0]:
print("Training set score: ", lin_reg.score(x_tr, y_tr))
print("Validation set score: ", lin_reg.score(x_val, y_val))

predicted_tr = lin_reg.predict(x_tr)
predicted_val = lin_reg.predict(x_val)

print("Training set MAE: ", mean_absolute_error(y_tr, predicted_tr))
print("validation set MAE: ", mean_absolute_error(y_val, predicted_val))

Training set score:  0.952966279796521
Validation set score:  -1.3820883352120975e+22
Training set MAE:  0.15228050052354156
validation set MAE:  3299119706.884302


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

## d) Usando Stump

Analizando la profundidad de los árboles, podemos hacer un experimento con profundidad 1. Calcularemos su score y también el MAE.

In [0]:
ini = time.time()
stump =  DecisionTreeRegressor(criterion='mae', max_depth=1)
stump.fit(x_tr, y_tr)
fin = time.time()
print(fin - ini, "[s]")

42.482816219329834 [s]


In [0]:
score_tr_stump = stump.score(x_tr, y_tr)
score_val_stump = stump.score(x_val, y_val)

print("Training score: ", stump.score(x_tr, y_tr))
print("Validation score: ", stump.score(x_val, y_val))

predicted_tr = stump.predict(x_tr)
predicted_val = stump.predict(x_val)
MAE_tr_stump = mean_absolute_error(y_tr, predicted_tr)
MAE_val_stump = mean_absolute_error(y_val, predicted_val)

print("Training set MAE: ", MAE_tr_stump)
print("Validation set MAE: ", MAE_val_stump)

Training score:  0.4097192095674549
Validation score:  0.40161242512700357
Training set MAE:  0.5823653752082486
Validation set MAE:  0.5882200215706899


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

Comparando valores, queda mejor expresado así:

In [0]:
print(pd.DataFrame([["Árbol profundidad 3", score_tr, MAE_tr], ["Stump", score_tr_stump, MAE_tr_stump]], columns = ["Tipo", "Score", "MAE"]))
print(pd.DataFrame([["Árbol profundidad 3", score_val, MAE_val], ["Stump", score_val_stump, MAE_val_stump]], columns = ["Tipo", "Score", "MAE"]))

                  Tipo     Score       MAE
0  Árbol profundidad 3  0.671262  0.373132
1                Stump  0.409719  0.582365
                  Tipo     Score       MAE
0  Árbol profundidad 3  0.668018  0.381435
1                Stump  0.401612  0.588220


La primera tabla es entrenamiento y la segunda es validación. Podemos ver que el desempeño de un árbol con más profundidad es mejor, esto debido a que toma más criterios a la hora de definir el target. Al tomar un solo nodo, se intenta dualizar la decisión respecto a un solo criterio, por lo que en casos más específicos tiende a equivocarse más. 

## e) Profundidad máxima

Debido a que el valor de la profundidad influye en el desempeño del árbol, realizaremos un KFold con los conjuntos para analizar como varía el desempeño mientras cambiamos el tamaño del árbol.

In [0]:
warnings.filterwarnings("ignore")

depth_tree = range(1, 21)
folds = KFold(5)
scores = np.zeros([20, 5])
mae = np.zeros([20, 5])
indexes =  []
for i in range(0, 20):
  j = 0
  for train_index, val_index in folds.split(x_tr):
    N = len(val_index)
    y_train_i = y_tr[train_index]
    x_train_fold = x_tr[train_index]
    y_val_i = y_tr[val_index]
    x_val_fold = x_tr[val_index]
    tree =  DecisionTreeRegressor(criterion='mae', max_depth = depth_tree[i])
    model = tree.fit(x_train_fold, y_train_i)
    scores[i][j] = model.score(x_val_fold,y_val_i)
    y_pred = model.predict(x_val_fold)
    mae[i][j] = np.sum([abs(y_pred[k] - y_val_i[k]) for k in range(N-1)])/N
    j+=1
    indexes.append((j, depth_tree[i]))
  print("Depth: ", depth_tree[i])

Depth:  1
Depth:  2
Depth:  3
Depth:  4
Depth:  5
Depth:  6
Depth:  7
Depth:  8
Depth:  9
Depth:  10
Depth:  11
Depth:  12
Depth:  13
Depth:  14
Depth:  15
Depth:  16
Depth:  17
Depth:  18
Depth:  19
Depth:  20


In [0]:
for i in range(0, 20):
  print("Depth: ", i + 1)
  print("    Average Validation Score: ", np.sum(scores[i]) / 5)
  print("    Average Validation MAE: ", np.sum(mae[i]) / 5)

Depth:  1
    Average Validation Score:  0.40703998684098674
    Average Validation MAE:  1.7497686282878049
Depth:  2
    Average Validation Score:  0.5710718646588967
    Average Validation MAE:  1.3795969951580758
Depth:  3
    Average Validation Score:  0.6701746197756391
    Average Validation MAE:  1.11888491861694
Depth:  4
    Average Validation Score:  0.7742793480296013
    Average Validation MAE:  0.873211987378923
Depth:  5
    Average Validation Score:  0.8389704089811321
    Average Validation MAE:  0.6654319509781759
Depth:  6
    Average Validation Score:  0.8947023702457425
    Average Validation MAE:  0.501087226914824
Depth:  7
    Average Validation Score:  0.9339391902600074
    Average Validation MAE:  0.36564722074138634
Depth:  8
    Average Validation Score:  0.9544865756002643
    Average Validation MAE:  0.2741450704326135
Depth:  9
    Average Validation Score:  0.9673924237235347
    Average Validation MAE:  0.2090677832394051
Depth:  10
    Average Validat

Podemos apreciar que mientras más aumenta la cantidad de hoja, el score del modelo va aumentando considerablemente. Esto claramente puede provocar overfitting, ya que si tenemos una gran cantidad de aciertos en un conjunto significa que está aprendiendo los patrones específicos de ese conjunto.

## f) Random Forest

Como se vió anteriormente, un árbol con mucha profundidad puede provocar overfitting y es algo inevitable, por lo que siempre se busca un nivel de profundidad donde podemos tener un mejor desempeño o, lo que se busca, generalización. 

Aparte de eso, podemos aplicar otro modelo llamado Random Forest, el cual genera una cantidad fija de árboles los cuales aprenden de partes parciales del conjunto de entrenamiento. Con esto en mente, cada árbol aprendió de partes distintas, por lo que tendrán una noción del espacio distinto. Esto implica a que dentro de diversas predicciones de nuevos datos, los árboles se desempeñaran de diversas formas, y la falencia que tenga en la predicción un árbol la puede complementar otro. Esto también se denomina un ensamble o un consenso entre los árboles, donde el modelo devuelve el valor del árbol con mejor desempeño.

Para probar esto, entrenaremos un modelo de Random Forest con una profundidad de 5, debido a que es un valor que en el KFold anterior era bastante bueno pero no tan cercano para definirlo como overfitting.

In [0]:
ini = time.time()
forest = RandomForestRegressor(n_estimators=20, criterion='mae', max_depth=5)
forest.fit(x_tr, y_tr)
fin = time.time()
print(fin - ini, "[s]")

964.1379325389862 [s]


In [0]:
score_tr_forest = forest.score(x_tr, y_tr)
score_val_forest = forest.score(x_val, y_val)

print("Training score: ", forest.score(x_tr, y_tr))
print("Validation score: ", forest.score(x_val, y_val))

predicted_tr_forest = forest.predict(x_tr)
predicted_val_forest = forest.predict(x_val)
MAE_tr_forest = mean_absolute_error(y_tr, predicted_tr_forest)
MAE_val_forest = mean_absolute_error(y_val, predicted_val_forest)

print("Training set MAE: ", MAE_tr_forest)
print("Validation set MAE: ", MAE_val_forest)

Training score:  0.8862274438695785
Validation score:  0.8894580319131108
Training set MAE:  0.21814603081047532
Validation set MAE:  0.2197568607296838


Podemos apreciar que el desempeño es bastante bueno, además no tiene una noción de overfitting comparado con árboles más profundos. Incluso, el desempeño de ambos conjuntos (entrenamiento y validación) poseen el mismo desempeño. Esto claramente porque con Random Forest se logra generalizar más el problema comparado a un árbol por sí solo.

## g) Otros parámetros

Por último, investigaremos más sobre los parámetros que tiene el árbol de decisión y como afectan en su desempeño, para poder buscar una forma de mejorar el desempeño.



*   Criterion: El criterio con el cual se calculará la función de pérdida, teniendo mse, mae y friedman_mse.
*   Splitter: La estrategia usada para decidir la partición en cada nodo, teniendo el criterio "best" para el mejor o "random".
*   max_depth: Es la máxima profundidad que el algoritmo buscará generar en el árbol
*   min_samples_split: Es la mínima cantidad de muestras en las cual se decidirá si se divide o no un nodo.
*   min_samples_leaf:
*   min_weight_fraction_leaf: 
*   max_features: El número de atributos que serán considerados para buscar la mejor división.
*   random_state: Es la semilla que genera números aleatorios. Si no se indica, se usa la semilla otorgada por numpy
*   max_leaf_nodes: 
*   min_impurity_decrease: El nodo se dividirá si, al hacer la división, la impureza es igual o menor a la actual (valor otorgado por este parámetro.)
*   min_impurity_split: Este valor actúa como umbral, donde un nodo se dividirá si supera este umbral o será una hoja en caso contrario.
*   presort: Variable booleana que reordena los datos de entrada.
